In [1]:
import pandas as pd
import ssl
import numpy as np
from pathlib import Path
import sqlalchemy
import datetime

database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

In [4]:
SectorDF = pd.read_sql_table('SectorDF', engine)

In [6]:
SectorDF.set_index('dateTime', inplace=True)
SectorDF

,SPY,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy
dateTime,,,,,,,,,,,,
2021-04-12 04:00:00,0.001898,0.004364,0.001239,0.000170,0.002117,0.006057,0.001120,-0.000095,0.006061,0.002487,0.002266,-0.017691
2021-04-13 04:00:00,0.003134,-0.001641,0.006489,0.000045,-0.000767,0.000146,-0.000002,0.016096,-0.004232,-0.003210,0.007684,0.001140
2021-04-14 04:00:00,-0.003415,-0.000804,0.000607,-0.004617,-0.004491,-0.001597,-0.002539,0.007733,0.010694,0.009282,-0.004979,0.020342
2021-04-15 04:00:00,0.005027,0.000453,0.011968,0.004855,0.003417,0.005316,-0.004535,0.011355,-0.001082,0.004324,0.011913,-0.006888
2021-04-16 04:00:00,-0.000072,-0.003498,0.002734,-0.002692,-0.003881,0.001754,0.002421,0.002735,-0.004246,-0.001499,-0.003487,-0.018957
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-06 04:00:00,-0.000783,0.002059,0.011836,-0.003392,-0.005008,0.009086,-0.006456,0.019110,-0.000604,-0.004417,0.014811,-0.010703
2022-04-07 04:00:00,0.006956,0.005856,0.020828,0.005208,-0.009337,0.008555,0.004695,-0.003217,-0.001480,0.004416,-0.004456,0.002626
2022-04-08 04:00:00,-0.001049,-0.005791,0.005329,-0.007141,0.001863,0.001890,0.004482,0.001318,0.002899,0.002020,0.001696,0.027976
